In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [21]:
data_dict = { 'image_id' : [],
              'true_class' : [],
              'predicted_class' : [],
               'inference_time' : [],
               'x' : [], 'y' : [], 'w' : [], 'h' : [], 
               'category' : []}

df = pd.DataFrame(data_dict)

In [22]:
path = 'C:/Users/TR1174/Desktop/Inference_Engine/persons'
files = os.listdir(path)
x = [os.path.splitext(f)[0] for f in files]
df['image_id'] = x

In [47]:
# #images not trained from dataset
# test_images = ['TCP_000074','TCP_000167','TCP_000220','TCP_000282','TCP_000361',
#                'TCP_st_000032', 'UCP_000036', 'UCP_000117', 'UCP_000199', 'UCP_000287'
#                 'UCP_st_000007']

# df['category'] = 'train'
# df.loc[[i in test_images for i in df.image_id], 'category'] = 'test'

df.loc[['Train' in i for i in df.image_id], 'category'] = 'train'
df['category'].fillna('val', inplace = True)

In [48]:
true_lables = []
for i in df.image_id:
    if 'TCP' in i:
        true_lables.append('person_phone_head')
    else:
        true_lables.append('person_phone_hand')
df.true_class = true_lables

# Predict

In [24]:
#put labels into csv
pred_labels = []
inference_time = []
x_, y_, w_, h_ = [], [], [], []

args = {
        "config": './yolov3-tiny.cfg',
        "weights": './yolov3-tiny_best_24k.weights',
        "classes": './object.names'
        }

def getOutputsNames(net):
    layersNames = net.getLayerNames()
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]


# Darw a rectangle surrounding the object and its class name 
def draw_pred(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])
    #print(label)
    pred_image.append(label)
    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

for i in df.image_id:    
    image = cv2.imread(path + '/' + i + '.jpg')
    x_t, y_t, w_t, h_t, = [], [], [], [] 
    pred_image = []
    
    classes = None
    with open(args['classes'], 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    #print(classes)

    COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

    # Define network from configuration file and load the weights from the given weights file
    net = cv2.dnn.readNet(args['weights'],args['config'])

    blob = cv2.dnn.blobFromImage(image, 1.0/255.0, (416,416), [0,0,0], True, crop=False) ###(416,416)

    #print('height..................',image.shape[0])
    Width = image.shape[1]
    Height = image.shape[0]
    net.setInput(blob)
    #print('net.............', net)
    outs = net.forward(getOutputsNames(net))
    #print('outs.........................................',outs)

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.3 ##################################0.5
    nms_threshold =  0.2 ##################################0.4


    #print(len(outs))

    # In case of tiny YOLOv3 we have 2 output(outs) from 2 different scales [3 bounding box per each scale]
    # For normal normal YOLOv3 we have 3 output(outs) from 3 different scales [3 bounding box per each scale]

    # For tiny YOLOv3, the first output will be 507x6 = 13x13x18
    # 18=3*(4+1+1) 4 boundingbox offsets, 1 objectness prediction, and 1 class score.
    # and the second output will be = 2028x6=26x26x18 (18=3*6) 

    for out in outs: 
        #print(out.shape)
        for detection in out:
            #print('detection....................',detection)
        #each detection  has the form like this [center_x center_y width height obj_score class_1_score class_2_score ..]
            scores = detection[5:]#classes scores starts from index 5
            #print(scores)
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            #print(confidence)
            if confidence >= 0.3:###############################0.5:
                #print(detection[0], Width)
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
    #print(boxes)
    # apply  non-maximum suppression algorithm on the bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    #print(indices)
    
    if indices == ():
        pred_image.append('none')
        x_t.append('none'), y_t.append('none'), w_t.append('none'), h_t.append('none')

    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        x_t.append(x)
        y = box[1]
        y_t.append(y)
        w = box[2]
        w_t.append(w)
        h = box[3]
        h_t.append(h)
        draw_pred(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    
    pred_labels.append(pred_image)
    x_.append(x_t), y_.append(y_t), w_.append(w_t), h_.append(h_t) 
    # Put efficiency information.
    t, _ = net.getPerfProfile()
    label = 'Inference time: %.2f ms' % (t * 1000.0 / cv2.getTickFrequency())
    inference_time.append('%.2f ms' % (t * 1000.0 / cv2.getTickFrequency()))
    cv2.putText(image, label, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0))

    #cv2.imwrite("Image.png", image)
    #plt.figure(figsize=(15,15))
    #plt.imshow(image)
    cv2.waitKey(0)


C:\Users\TR1174\AppData\Local\Continuum\anaconda3\envs\pd\lib\site-packages\ipykernel_launcher.py:96: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [25]:
len(pred_labels), len(inference_time), len(x_), len(y_)

(224, 224, 224, 224)

In [26]:
df['predicted_class'] = pred_labels
df['inference_time'] = inference_time
df['x'] = x_
df['y'] = y_ 
df['w'] = w_ 
df['h'] = h_

In [27]:
pd.set_option('display.max_rows', 1500)
df

,image_id,true_class,predicted_class,inference_time,x,y,w,h,category
0,ROI_0001,NaN,[person_phone_head],121.11 ms,[-11.0],[16.5],[90],[113],NaN
1,ROI_0002,NaN,[person_phone_head],108.26 ms,[-13.0],[47.0],[92],[136],NaN
2,ROI_0003,NaN,[person_phone_head],130.93 ms,[9.0],[6.0],[60],[78],NaN
3,ROI_0006,NaN,[none],133.40 ms,[none],[none],[none],[none],NaN
4,ROI_0007,NaN,[person_phone_hand],138.03 ms,[12.0],[7.0],[48],[42],NaN
5,ROI_0008,NaN,[none],132.49 ms,[none],[none],[none],[none],NaN
6,ROI_0009,NaN,[person_phone_head],113.90 ms,[-15.0],[38.0],[112],[130],NaN
7,ROI_0011,NaN,[person_phone_hand],86.72 ms,[5.0],[11.0],[48],[84],NaN
8,ROI_0012,NaN,[person_phone_hand],145.83 ms,[-5.0],[16.5],[88],[81],NaN
9,ROI_0013,NaN,[person_phone_head],142.80 ms,[10.0],[5.5],[60],[79],NaN


In [10]:
df.to_csv('Infernece_tiny_240_epochs_thresh_0.5.csv', index = False)

In [47]:
df[df.category == 'train'].shape[0], df[df.category == 'val'].shape[0]

(575, 143)